In [1]:
# import essentials
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import tensorflow as tf
import random

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
from google.colab import files
import json

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed = 69
seed_everything(seed)

In [6]:
train = pd.read_csv('/content/gdrive/My Drive/input (1)/moa/train_features.csv')
target = pd.read_csv('/content/gdrive/My Drive/input (1)/moa/train_targets_scored.csv')

print(f'train shape: {train.shape}')
print(f'target shape: {target.shape}')
train.head()

train shape: (23814, 876)
target shape: (23814, 207)


,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,0.2198,...,0.4805,0.4965,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,-0.2102,...,0.4083,0.0319,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,-1.1800,...,-0.5477,-0.7576,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,-1.6740,...,-2.1220,-0.3752,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,-0.7110,...,-0.2274,0.3215,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [7]:
target.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
clean_train = train.drop(columns=['sig_id'])
clean_train['vehicle'] = train.cp_type.apply(
    lambda x: x == 'ctl_vehicle'
).astype(np.int8)
clean_train.drop(columns=['cp_type'],inplace=True)
clean_train.vehicle.value_counts()

0    21948
1     1866
Name: vehicle, dtype: int64

In [9]:
clean_target = target.drop(columns=['sig_id'])

In [10]:
train.cp_time.value_counts()

48    8250
72    7792
24    7772
Name: cp_time, dtype: int64

In [11]:
train.cp_dose.value_counts()

D1    12147
D2    11667
Name: cp_dose, dtype: int64

In [12]:
clean_train['d2'] = train.cp_dose.apply(
    lambda x: x == 'D2'
).astype(np.int8)
clean_train.drop(columns=['cp_dose'],inplace=True)
clean_train.shape

(23814, 875)

In [13]:
# For a more general case, we could implement this code:
from sklearn.preprocessing import LabelEncoder
def convert_cat_cols(df :pd.DataFrame) :
    """

    """
    le = LabelEncoder()
    new_df = df.copy()
    for col in df.keys() :
        if np.issubdtype(df[col].dtype, np.number) : continue
        # check if it's a unique non-numeric identifier
        if df[col].nunique() == df.shape[0] : 
            new_df.drop(columns=[col],inplace=True)
        # it isnt a number or a unique identifier, so convert it to a label
        new_df[col] = le.fit_transform(df[col])
    return new_df

In [14]:
# that cleans up the X matrix. Now for the Y matrix
convert_cat_cols(train).head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,...,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99,sig_id
0,1,24,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,0.2198,0.2965,...,0.4965,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,0
1,1,72,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,-0.2102,0.1656,...,0.0319,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371,1
2,1,48,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,-1.1800,0.1256,...,-0.7576,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,2
3,1,48,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,-1.6740,-1.6720,...,-0.3752,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,3
4,1,72,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,-0.7110,-1.4470,...,0.3215,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,4


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
# let's check for balance
clean_target = target.drop(columns='sig_id')
clean_target.mean().mean()

0.003433568081433697

In [17]:
# very unbalanced. We may use metrics other than accuracy to judge our interim results
# however, the final judgment is based on log loss of liklihood, so we will ultimately 
# compare this to a log loss estimate

In [18]:
'''
corrmat = pd.concat([clean_train,target.iloc[:,1]],axis=1).corr(method='spearman')['5-alpha_reductase_inhibitor']\
    .drop('5-alpha_reductase_inhibitor').sort_values(ascending=False)
fig = plt.figure(figsize=(10,12))
plt.title('Spearman correlation with first target')
fig = sns.barplot(y=corrmat.index,x=corrmat.values,
                  palette='twilight_shifted',orient='h')
'''

"\ncorrmat = pd.concat([clean_train,target.iloc[:,1]],axis=1).corr(method='spearman')['5-alpha_reductase_inhibitor']    .drop('5-alpha_reductase_inhibitor').sort_values(ascending=False)\nfig = plt.figure(figsize=(10,12))\nplt.title('Spearman correlation with first target')\nfig = sns.barplot(y=corrmat.index,x=corrmat.values,\n                  palette='twilight_shifted',orient='h')\n"

In [19]:
# import our evaluation methods
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, log_loss, roc_auc_score
from sklearn.model_selection import cross_val_score

In [20]:
# import oversampling techniques
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [21]:
def score(y_test,y_pred,method='log_loss') :
    if method == 'log_loss' : return log_loss(y_test,y_pred)
    if method == 'roc' : return roc_auc_score(y_test,y_pred)

In [22]:
# import our models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [23]:
# import our ensambling methods
from imblearn.pipeline import Pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.multioutput import MultiOutputClassifier

In [24]:
def make_pipe(clf,y) :
    num_samples = y.sum() * .8 - 1
    pipe = Pipeline([
        ('scaler',StandardScaler()),
        ('sampling',SMOTE(k_neighbors=max(5,int(y.sum()*.8-1)))),
        ('clf',clf)
    ])
    return pipe

In [25]:
def non_smote_pipe(clf,y) :
    pipe = Pipeline([
        ('scaler',StandardScaler()),
        ('clf',clf)
    ])
    return pipe

In [26]:
def fit_score_pipe(pipe,X,y) :
    X_train,X_test,y_train,y_test = train_test_split(
        X,y,stratify=y,
        test_size=.2,random_state=seed)
    pipe.fit(X_train.copy(),y_train.copy())
    y_pred = pipe.predict_proba(X_test)
    return log_loss(y_test,y_pred,labels=[0,1])

In [27]:
def single_sample_score_pipe(pipe,df,target,col) :
    n = 8
    repeat_rows = df[target[col]==1]
    repeat_target = target[target[col]==1][col]

    dup_df = df.copy()
    dup_target = pd.Series(target[col])
    for i in range(n) :
        dup_df = pd.concat([dup_df,repeat_rows])
        dup_target = dup_target.append(repeat_target)

    X_train,X_test,y_train,y_test = train_test_split(
        dup_df,dup_target,test_size=.2,
        stratify=dup_target,
        random_state=seed)
    
    

    pipe.fit(X_train.copy(),y_train.copy())
    y_pred = pipe.predict_proba(X_test)
    return log_loss(y_test,y_pred,labels=[0,1])

In [28]:
def pipe_gridsearch(pipe,param_grid=None) :
    pass

In [38]:
lgbm_params = {
    'num_leaves': 491,
    'min_child_weight': 0.03,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.4,
    'min_data_in_leaf': 106,
    'objective': 'binary',
    'max_depth': -1,
    'learning_rate': 0.005,
    "boosting_type": "gbdt",
    "bagging_seed": seed,
    "metric": 'binary_logloss',
    "verbosity": 0,
    'reg_alpha': 0.4,
    'reg_lambda': 0.6,
    'random_state': seed
}

In [39]:
clf = LGBMClassifier(**lgbm_params)

In [31]:
def repeat_sample(X,y,n=6) :
    repeat_rows = X[y==1]
    repeat_target = y[y==1]

    dup_df = X.copy()
    dup_target = y.copy()
    for i in range(n) :
        dup_df = pd.concat([dup_df,repeat_rows])
        dup_target = dup_target.append(repeat_target)
    
    return dup_df,dup_target

In [77]:
def make_pipe_dict(pipe_dict,clf,df,Y) :
    loss_dict = {}
    scores = 0
    models = []
    for i,col in enumerate(Y.keys()) :
        if col == 'sig_id' : continue # unnecessary since we dropped it, but in case we forget on the test set...
        if col in pipe_dict : 
            continue
        start_time = time()

        X = df.copy()
        y = Y[col].copy()

        pipe = make_pipe(clf,y)

        if y.sum() > 6 :
            loss = fit_score_pipe(pipe,X,y)
        else :
            print(f'\tToo few samples in {col}. Number of positive examples: {y.sum()}')
            
            pipe = non_smote_pipe(clf,y)
            
            
            if Y[col].sum() < 2 : 
                print('\tOnly 1 class example found. Increasing Samples')
                X,y = repeat_sample(X,y)

        loss = fit_score_pipe(pipe,X,y)
        scores += loss

        print('{}\t\t{}\t\t{:.5f}'
            .format(str(datetime.timedelta(seconds=time() - start_time))[:7],
                col, loss))

        pipe_dict[col] = pipe
        loss_dict[col] = loss
        
        if i % 20 == 0 :
            with open("models.pckl", "wb") as f:
                for model in models:
                    pickle.dump(model, f)
            models = []

    scores /= len(Y.keys())
    print(f'Score: {scores}')
    return pipe_dict,loss_dict

In [90]:
from time import time
import datetime
import pickle

In [91]:
with open('/content/gdrive/My Drive/input (1)/moa/pipe_dict.pkl','rb') as handle :
    pipe_dict = pickle.load(handle)

In [92]:
t = time()
lgbm = LGBMClassifier()
pipe_dict,loss_dict = make_pipe_dict(pipe_dict,lgbm,clean_train,clean_target)

print('\nTime elapsed: {:.1f}s'.format(time() - t))

0:01:10		11-beta-hsd1_inhibitor		0.00944
Score: 4.583138318608527e-05

Time elapsed: 70.1s


In [93]:
with open('/content/gdrive/My Drive/input (1)/moa/pipe_dict.pkl','wb') as handle :
    pickle.dump(pipe_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [94]:
clean_target.sum(axis=0).sort_values(ascending=True)

atp-sensitive_potassium_channel_antagonist      1
erbb2_inhibitor                                 1
diuretic                                        6
autotaxin_inhibitor                             6
protein_phosphatase_inhibitor                   6
                                             ... 
serotonin_receptor_antagonist                 404
dopamine_receptor_antagonist                  424
cyclooxygenase_inhibitor                      435
proteasome_inhibitor                          726
nfkb_inhibitor                                832
Length: 206, dtype: int64

In [95]:
clean_target.shape

(23814, 206)

In [96]:
clean_train.shape

(23814, 875)

In [97]:
Y = clean_target
Y.laxative.sum()

6

In [98]:
import tensorflow
from tensorflow.keras import layers


In [99]:
def create_model() :
    pass

In [100]:
feat_dict = {}
for k,model in pipe_dict.items() :
    importances = pipe_dict[k].named_steps['clf'].feature_importances_
    
    feat_dict[k] = [x for _,x in sorted(zip(importances,clean_target.keys()))]
    print(k,feat_dict[k][:5])

5-alpha_reductase_inhibitor ['5-alpha_reductase_inhibitor', 'acat_inhibitor', 'acetylcholine_receptor_agonist', 'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist']
11-beta-hsd1_inhibitor ['adenosine_receptor_antagonist', 'adrenergic_receptor_agonist', 'aldehyde_dehydrogenase_inhibitor', 'alk_inhibitor', 'ampk_activator']
